## 尝试加速MLD算法

我们发现虽然每个detector所连接的边和节点都是有限的，由于计算分布的规模随着边数量的增加而指数增加，因此依旧是一个比较大的数，最大可能为：
$$2^{81}$$
这是一个很大的数，因此我们需要加速算法。

我们考虑一个d=5，r=7的示例。

In [1]:
import MLD
import stim

In [2]:
circuit_noisy = stim.Circuit.from_file("./surface_code_bZ_d5_r07_center_5_5/circuit_noisy.stim")

detector_error_model = circuit_noisy.detector_error_model()

In [3]:
ml_decoder = MLD.MaxLikelihoodDecoder(detector_error_model,detector_error_model.num_detectors)
ml_decoder.compute_detector_number_distribution()

{'D0': 19,
 'D3': 11,
 'D17': 55,
 'D4': 22,
 'D13': 46,
 'D16': 58,
 'D18': 59,
 'D19': 68,
 'L0': 207,
 'D1': 19,
 'D23': 70,
 'D5': 11,
 'D22': 48,
 'D26': 16,
 'D27': 50,
 'D2': 3,
 'D6': 22,
 'D14': 28,
 'D20': 56,
 'D24': 70,
 'D7': 22,
 'D28': 71,
 'D30': 53,
 'D8': 3,
 'D33': 18,
 'D9': 10,
 'D25': 43,
 'D10': 19,
 'D29': 56,
 'D31': 54,
 'D11': 8,
 'D34': 38,
 'D21': 26,
 'D32': 20,
 'D35': 10,
 'D12': 20,
 'D41': 68,
 'D37': 52,
 'D40': 64,
 'D36': 22,
 'D15': 28,
 'D42': 68,
 'D43': 80,
 'D44': 64,
 'D38': 28,
 'D47': 80,
 'D46': 56,
 'D39': 28,
 'D48': 80,
 'D49': 56,
 'D51': 64,
 'D50': 26,
 'D52': 80,
 'D53': 68,
 'D45': 26,
 'D54': 68,
 'D55': 64,
 'D56': 28,
 'D57': 28,
 'D58': 52,
 'D59': 22,
 'D65': 68,
 'D61': 52,
 'D64': 64,
 'D60': 22,
 'D66': 68,
 'D67': 80,
 'D68': 64,
 'D62': 28,
 'D71': 80,
 'D70': 56,
 'D63': 28,
 'D72': 80,
 'D73': 56,
 'D75': 64,
 'D74': 26,
 'D76': 80,
 'D77': 68,
 'D69': 26,
 'D78': 68,
 'D79': 64,
 'D80': 28,
 'D81': 28,
 'D82': 52,
 'D83

## 假设我们已知D43的值为0，对于D43的相关项，我们应该怎么处理？

In [4]:
D43_error_detector_model = {}

for error in detector_error_model:
    if error.type == "error":
        targets_copy = error.targets_copy()
        fliped_detector_observable_index = ml_decoder.get_detector_logical_observable_val(targets_copy, ml_decoder.detector_number)
        if 43 in fliped_detector_observable_index:
            probability = error.args_copy()[0]
            D43_error_detector_model[tuple(fliped_detector_observable_index)] = probability
            print(error)

error(0.00180976) D13 D16 D17 D43
error(0.000500323) D13 D16 D40 D43
error(0.00130303) D13 D16 D41 D43
error(0.000457248) D13 D17 D41 D43
error(0.00153124) D13 D40 D41 D43
error(0.00367973) D13 D43
error(0.000457248) D14 D16 D17 D43
error(0.000457248) D14 D16 D41 D43
error(0.00131259) D14 D17 D41 D43
error(0.000577662) D14 D17 D43
error(0.000577662) D14 D41 D43
error(0.00131259) D14 D43
error(0.00163785) D16 D17 D19 D43
error(0.00025984) D16 D17 D20 D43
error(0.000339969) D16 D19 D43 D47
error(0.000970795) D16 D19 D43 D48
error(0.00025984) D16 D20 D43 D48
error(0.00122997) D16 D40 D42 D43
error(0.000500323) D16 D41 D42 D43
error(0.00116262) D16 D42 D43 D47
error(0.000339969) D16 D42 D43 D48
error(0.00025984) D17 D19 D43 D48
error(0.00131677) D17 D20 D43 D48
error(0.000320939) D17 D20 D43 D49
error(0.00217206) D17 D41 D43 D44
error(0.000577662) D17 D43 D44
error(0.00237288) D17 D43 D44 D48
error(0.000320939) D17 D43 D44 D49
error(0.0320931) D19 D43
error(0.00126654) D19 D43 D47 D48
erro

多个错误事件同时发生的概率其实不大。

In [5]:
import itertools
probabilities = sorted(D43_error_detector_model.values(), reverse=True)

In [6]:
def get_i_error_even_probability(probabilities: list, i: int):
    """i个错误事件同时发生的概率

    Args:
        probabilities (list): 当前detector对应的所有错误事情
        i (int): i个错误事件发生
    """
    i_error_evens = itertools.combinations(range(len(probabilities)), i)
    i_error_even_probabilities = 0
    for error_even_index in i_error_evens:
        prob = 1
        for prob_index in range(len(probabilities)):
            if prob_index in error_even_index:
                prob = prob * probabilities[prob_index]
            else:
                prob = prob * (1 - probabilities[prob_index])
        i_error_even_probabilities += prob
    return i_error_even_probabilities

In [7]:
for i in range(1,5):
    i_probability = get_i_error_even_probability(probabilities=probabilities, i=i)
    print(f"{i}个错误事件同时发生的概率为{i_probability}")

1个错误事件同时发生的概率为0.18559936361848192
2个错误事件同时发生的概率为0.020225557658935214
3个错误事件同时发生的概率为0.0013692091981292408
4个错误事件同时发生的概率为6.485067761072154e-05


假设，我们不考虑，错误事件组合，使得发生10^{-15}的概率。

In [8]:
probabilities[0]*probabilities[1]*probabilities[2]*probabilities[3]*probabilities[4]*probabilities[5]*probabilities[6]*probabilities[7]

8.104974546600419e-15

假设为1，则可能的组合是1,3,5,7.
假设为0，则可能的组合为0,2,4,6.

上面设置的10^{-15}次方，和其他时间发生，值应该更小，但是也存在一些可以合并的情况，可能对应会增加。

这个值如何设置，或者设置这个值是否会带来误差，需要进行思考。

因为，本身是probability存在顺序，因此我们可以有一定的优化：

In [9]:
probabilities

[0.03209314439133004,
 0.03209314439133004,
 0.018217423062421856,
 0.014759755055715348,
 0.014566244437595306,
 0.014566244437595306,
 0.011744456314717388,
 0.011744456314717388,
 0.011440706458813104,
 0.008704607463930172,
 0.003679729432590249,
 0.0035765165723576037,
 0.002372880000104484,
 0.0021720613961547534,
 0.0018097649976214288,
 0.001642474710774992,
 0.0016378526356232723,
 0.0016378526356232723,
 0.001553108689026702,
 0.001541012847976231,
 0.0015312353629818934,
 0.001494935687764309,
 0.0014402115842943254,
 0.0013167694462953802,
 0.0013167694462953802,
 0.0013125946963400423,
 0.0013125946963400423,
 0.0013030340896023013,
 0.001277933125358692,
 0.0012665431220169816,
 0.0012665431220169816,
 0.0012299675340994892,
 0.00116261892345543,
 0.001106268851653764,
 0.0010206214357338142,
 0.0010206214357338142,
 0.001012104479087138,
 0.001012104479087138,
 0.0009707951757025881,
 0.0009707951757025881,
 0.0005776624679265652,
 0.0005776624679265652,
 0.0005776624679

* 基于具体的测量值，为0则只能是偶数个错误事件发生，为1只能是奇数个错误事件发生。
* 错误翻转是由几个错误事件发生而来。（多个错误事件发生，对应的概率会比较小）

因此，我们可以考虑设置一个阈值，选择出概率大于阈值的错误事件组合。

这个后续继续考虑

## d=3, r=3的并行在线MLD示例

In [10]:
import MLD
import stim

circuit_noisy = stim.Circuit.from_file("./surface_code_bZ_d3_r03_center_3_5/circuit_noisy.stim")

detector_error_model = circuit_noisy.detector_error_model()

在均匀的噪声模型下，得到的基本情况为：
```
L0:35
max_key:D9, max_value:48
connected_detector len:12
max connectivity detector coordinates: [2.0, 4.0, 1.0]
max connectivity detector: D9, detector indexs: [2, 5, 6, 8, 10, 13, 24, 12, 16, 7, 14, 11, 17], detector indexs len: 13
max connectivity detector's connected detector coordinates: [[4.0, 4.0, 0.0], [2.0, 2.0, 1.0], [4.0, 2.0, 1.0], [0.0, 4.0, 1.0], [4.0, 4.0, 1.0], [2.0, 2.0, 2.0], [2.0, 0.0, 2.0], [0.0, 4.0, 2.0], [6.0, 2.0, 1.0], [4.0, 2.0, 2.0], [4.0, 6.0, 1.0], [2.0, 4.0, 2.0]]
detector_error_model_number:24
```

意思是，在d=3, r=3的模型下，存在24个检测器（detector），其中最大连接数量的detector为D9，其相连的边数为48，其相连的detector数为13（因为是超图，所以连接边数!=连接节点数）。

D9对应的坐标为[2.0, 4.0, 1.0]，其连接的detector为[[4.0, 4.0, 0.0], [2.0, 2.0, 1.0], [4.0, 2.0, 1.0], [0.0, 4.0, 1.0], [4.0, 4.0, 1.0], [2.0, 2.0, 2.0], [2.0, 0.0, 2.0], [0.0, 4.0, 2.0], [6.0, 2.0, 1.0], [4.0, 2.0, 2.0], [4.0, 6.0, 1.0], [2.0, 4.0, 2.0]].

In [11]:
detector_error_model

stim.DetectorErrorModel('''
    error(0.00538601) D0 D2
    error(0.00455203) D0 D2 D8
    error(0.00341035) D0 D3
    error(0.0287947) D0 D5
    error(0.000418444) D0 D5 D7
    error(0.00163539) D0 D5 D7 D8
    error(0.00232627) D0 D5 D8
    error(0.000500323) D0 D7 D8 D9
    error(0.00153124) D0 D7 D8 D10
    error(0.000418444) D0 D7 D8 L0
    error(0.00167586) D0 D7 D9
    error(0.000500323) D0 D7 D10
    error(0.00480341) D0 D7 L0
    error(0.000500323) D0 D8 D9
    error(0.00175909) D0 D8 D10
    error(0.000418444) D0 D8 L0
    error(0.00167586) D0 D9
    error(0.00547618) D0 D10
    error(0.00897529) D0 L0
    error(0.00400415) D1 D3
    error(0.0299506) D1 D9
    error(0.00278521) D1 L0
    error(0.00724199) D2
    error(0.00244636) D2 D5
    error(0.00244636) D2 D5 D8
    error(0.0231764) D2 D6
    error(0.00218788) D2 D6 D8
    error(0.00244473) D2 D8
    error(0.00234426) D2 D8 D10
    error(0.00234426) D2 D10
    error(0.0121274) D3
    error(0.00257744) D3 D9
    error(0.00

我们发现上面的错误事件数为219个。

我们先按照detector的序列顺序，执行在线的MLD算法。

In [12]:
ml_decoder = MLD.MaxLikelihoodDecoder(detector_error_model,detector_error_model.num_detectors)

ml_decoder.compute_detector_number_distribution()

{'D0': 19,
 'D2': 10,
 'D8': 43,
 'D3': 8,
 'D5': 45,
 'D7': 48,
 'D9': 16,
 'D10': 34,
 'L0': 35,
 'D1': 3,
 'D6': 26,
 'D11': 10,
 'D4': 20,
 'D16': 48,
 'D13': 34,
 'D15': 43,
 'D12': 10,
 'D18': 45,
 'D17': 26,
 'D14': 16,
 'D19': 20,
 'D20': 8,
 'D23': 19,
 'D21': 10,
 'D22': 3}